# KOSPI 반도체 섹터 기업 주가, 거래량 크롤링 from Naver Finance

증권사 API를 쓰지 않고 크롤링으로 주가 데이터를 가져올 때 수정종가(Adjusted Close)를 가져오는 것이 큰 난관이다. 

네이버의 일별시셰 페이지는 단순 시가를 보여주고 수정종가를 보여주지 않기 때문에 네이버의 차트 페이지의 xml페이지를 불러 데이터를 가져와야 한다. 

검색해보니 종전에는 다음(카카오) 파이넌스에서 수정종가를 제공하여 크롤링을 많이 했다고 하는데, 현재는 http request를 통한 시세 페이지 접근을 막은 것으로 보인다. 

http://henryquant.blogspot.com/2018/11/r.html 를 참고하였다. 

In [3]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs

In [54]:
ticker = ''
daycount = 570 # from 20161227 ~ 20190429
stock_xml_url = "https://fchart.stock.naver.com/sise.nhn?symbol={ticker}&timeframe=day&count={daycount}&requestType=0".format(ticker=ticker, daycount=daycount) 

우선 네이버 파이넌스의 '반도체와 반도체장비' 업종 내 종목들 중 KOSPI 종목들만 ticker을 가져온다. 

페이지의 html 내에서 ticker는 쉽게 가져올 수 있지만 KOSPI/KOSDAQ 여부는 조금 번거롭지만 종목명 뒤에 *이 달려있는지 달려있지 않은지를 통해 필터해야 한다. 

In [55]:
semiconductor_html = urlopen("https://finance.naver.com/sise/sise_group_detail.nhn?type=upjong&no=202")
semiconductor_bs = bs(semiconductor_html.read(), 'html.parser')

In [56]:
ticker_list = []

for tr in semiconductor_bs.find('table', {'summary':'업종별 시세 리스트'}).find_all('tr')[1:]:
    if not tr.td.span:
        continue
    
    if tr.td.span.get_text() != '*': # 코스닥 종목은 *. 코스피는 *가 없다. 
        ticker_list.append(tr.td.a.attrs['href'])

ticker_list = [x[-6:] for x in ticker_list] # Naver의 반도체와 반도체 장비 업종 내 KOSPI 종목들 ticker리스트. eg: 003160

In [57]:
ticker_list

['003160',
 '077500',
 '281820',
 '000990',
 '005930',
 '042700',
 '000660',
 '029460',
 '000995',
 '005935',
 '195870',
 '092220',
 '006200',
 '025560']